# OpenAI Introduction

Use this for [reference](https://platform.openai.com/docs/api-reference/authentication)

In [ ]:
%pipenv install openai python-dotenv -q

> Create an OpenAI account and get an API_KEY

### OpenAI:

1. Chat completion API ✅
2. Implement structured output ✅
3. Implement tool calling ✅
4. Implement streaming ✅
5. Implement assistant ✅
6. Response API ✅
7. Chaining of response ✅

In [1]:
from openai import AzureOpenAI
from dotenv import load_dotenv
from os import getenv

load_dotenv()

True

In [2]:
client = AzureOpenAI(
    api_key=getenv("AZURE_OPENAI_API_KEY"),
    azure_endpoint=getenv("AZURE_OPENAI_ENDPOINT"),
    api_version=getenv("AZURE_OPENAI_API_VERSION")
)

### List Models

In [3]:
models = client.models.list()
model_names = [model.id for model in models if "gpt" in model.id]

In [4]:
model_names[:5]

['gpt-35-turbo-0301',
 'gpt-35-turbo-0613',
 'gpt-35-turbo-1106',
 'gpt-35-turbo-0125',
 'gpt-35-turbo-instruct-0914']

| 1. Chat Completion |
|--|


The Chat Completions API endpoint will generate a model response from a list of messages comprising a conversation.

In [5]:
MODEL_NAME = "gpt-4o-mini"

In [6]:
completion = client.chat.completions.create(
  model=MODEL_NAME,
  store=False,
  messages=[
    {"role": "user", "content": "write a haiku about ai"}
  ]
)

In [7]:
print(completion.choices[0].message.content)

Silent thoughts in code,  
Minds of circuits weave and learn,  
Dreams without a soul.


| 2. Structured Output |
|--|

Structured Outputs is a feature that ensures the model will always generate responses that adhere to your supplied JSON Schema,

In [ ]:
!pipenv install pydantic

In [7]:
from pydantic import BaseModel

In [8]:
class UserRegistration(BaseModel):
    name: str
    age: int
    tier: int

In [ ]:
completion = client.beta.chat.completions.parse(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": "Extract the user information."},
        {"role": "user", "content": "We have a new user named Tim aged 25 with a tier 3 account."},
    ],
    response_format=UserRegistration,
)

In [ ]:
event = completion.choices[0].message.parsed
event

`json_schem` is an alternate method for passing schema

In [18]:
from pprint import pprint
from json import loads

In [ ]:
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": "You are a helpful math tutor. Guide the user through the solution step by step."},
        {"role": "user", "content": "how can I solve 8x + 7 = -23"}
    ],
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "math_response",
            "schema": {
                "type": "object",
                "properties": {
                    "steps": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "explanation": {"type": "string"},
                                "output": {"type": "string"}
                            },
                            "required": ["explanation", "output"],
                            "additionalProperties": False
                        }
                    },
                    "final_answer": {"type": "string"}
                },
                "required": ["steps", "final_answer"],
                "additionalProperties": False
            },
            "strict": True
        }
    }
)

In [19]:
pprint(loads(response.choices[0].message.content), indent=4)

{   'final_answer': 'x = -3.75 or x = -15/4',
    'steps': [   {   'explanation': 'Start with the original equation: 8x + 7 '
                                    '= -23.',
                     'output': '8x + 7 = -23'},
                 {   'explanation': 'Subtract 7 from both sides to isolate the '
                                    'term with x.',
                     'output': '8x = -23 - 7'},
                 {   'explanation': 'Now simplify the right side: -23 - 7 = '
                                    '-30.',
                     'output': '8x = -30'},
                 {   'explanation': 'Next, divide both sides by 8 to solve for '
                                    'x.',
                     'output': 'x = -30 / 8'},
                 {   'explanation': 'Now simplify the fraction: -30 / 8 = -15 '
                                    '/ 4 or -3.75.',
                     'output': 'x = -3.75 or x = -15/4'}]}


Streaming with Structured Output (JSON Key)

In [25]:
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": "You are a helpful math tutor. Guide the user through the solution step by step."},
        {"role": "user", "content": "how can I solve 8x + 7 = -23"}
    ],
    stream=True,
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "math_response",
            "schema": {
                "type": "object",
                "properties": {
                    "steps": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "explanation": {"type": "string"},
                                "output": {"type": "string"}
                            },
                            "required": ["explanation", "output"],
                            "additionalProperties": False
                        }
                    },
                    "final_answer": {"type": "string"}
                },
                "required": ["steps", "final_answer"],
                "additionalProperties": False
            },
            "strict": True
        }
    }
)

cnt = 0
for chunk in response:
    print(chunk)
    print("****************")
    
    if cnt == 10:
        break
    cnt += 1

ChatCompletionChunk(id='', choices=[], created=0, model='', object='', service_tier=None, system_fingerprint=None, usage=None, prompt_filter_results=[{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}])
****************
ChatCompletionChunk(id='chatcmpl-Bbn2Q5h0aZ5Lqu7Azp9Bg4tdvUw50', choices=[Choice(delta=ChoiceDelta(content='', function_call=None, refusal=None, role='assistant', tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1748346338, model='gpt-4o-mini-2024-07-18', object='chat.completion.chunk', service_tier=None, system_fingerprint='fp_7a53abb7a2', usage=None)
****************
ChatCompletionChunk(id='chatcmpl-Bbn2Q5h0aZ5Lqu7Azp9Bg4tdvUw50', choices=[Choice(delta=ChoiceDelta(content='{"', function_call=N

> Simply passing `stream` to `True` for the Completion api does not work

Most if not all solutions this involves writing our own parser. I'll be trying with the `openai-streaming` package

In [57]:
from typing import List

In [56]:
class EntitiesModel(BaseModel):
    attributes: List[str]
    colors: List[str]
    animals: List[str]

In [61]:
with client.beta.chat.completions.stream(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": "Extract entities from the input text"},
        {
            "role": "user",
            "content": "The quick brown fox jumps over the lazy dog with piercing blue eyes",
        },
    ],
    response_format=EntitiesModel,
) as stream:
    for event in stream:
        if event.type == "content.delta":
            if event.parsed is not None:
                # Print the parsed data as JSON
                print("content.delta parsed:", event.parsed)
        elif event.type == "content.done":
            print("content.done")
        elif event.type == "error":
            print("Error in stream:", event.error)

final_completion = stream.get_final_completion()
print("Final completion:", final_completion)

content.delta parsed: {}
content.delta parsed: {}
content.delta parsed: {'attributes': []}
content.delta parsed: {'attributes': []}
content.delta parsed: {'attributes': ['quick']}
content.delta parsed: {'attributes': ['quick']}
content.delta parsed: {'attributes': ['quick', 'brown']}
content.delta parsed: {'attributes': ['quick', 'brown']}
content.delta parsed: {'attributes': ['quick', 'brown', 'lazy']}
content.delta parsed: {'attributes': ['quick', 'brown', 'lazy']}
content.delta parsed: {'attributes': ['quick', 'brown', 'lazy']}
content.delta parsed: {'attributes': ['quick', 'brown', 'lazy', 'piercing']}
content.delta parsed: {'attributes': ['quick', 'brown', 'lazy', 'piercing']}
content.delta parsed: {'attributes': ['quick', 'brown', 'lazy', 'piercing', 'blue']}
content.delta parsed: {'attributes': ['quick', 'brown', 'lazy', 'piercing', 'blue']}
content.delta parsed: {'attributes': ['quick', 'brown', 'lazy', 'piercing', 'blue']}
content.delta parsed: {'attributes': ['quick', 'brown'

| 3. Tool/Function Calling |
|--|

Function calling provides a powerful and flexible way for OpenAI models to interface with your code or external services. 



In [ ]:
tools = [{
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current temperature for a given location.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "City and country e.g. Bogotá, Colombia"
                }
            },
            "required": [
                "location"
            ],
            "additionalProperties": False
        },
        "strict": True
    }
}]

| Function -> Tool JSON Convertor |
|--|

Automate tool JSON creation

In [ ]:
import inspect
from typing import Callable, get_type_hints

In [ ]:
def generate_json_schema(func: Callable) -> str:
    """
    Generate a JSON schema for a given function based on its docstring and type hints.

    Args:
        func (Callable): The Python function to generate a schema for.

    Returns:
        str: A JSON string representing the function's schema.

    Raises:
        ValueError: If the function lacks a docstring.
    """
    # Get function metadata
    if not func.__doc__:
        raise ValueError("Function must have a docstring")

    signature = inspect.signature(func)
    type_hints = get_type_hints(func)

    # Parse docstring
    docstring = inspect.getdoc(func) or ""
    doc_lines = docstring.split('\n')
    description = doc_lines[0].strip()  # First line is the main description
    param_descriptions = {}
    in_args_section = False

    # Extract parameter descriptions
    for line in doc_lines:
        line = line.strip()
        if line.lower().startswith('args:'):
            in_args_section = True
            continue
        if in_args_section and line and ':' in line:
            # Handle lines like "param_name (type): description"
            parts = line.split(':', 1)
            param_part = parts[0].strip()
            desc = parts[1].strip() if len(parts) > 1 else ""
            # Extract param_name from "param_name (type)"
            if '(' in param_part and ')' in param_part:
                param_name = param_part[:param_part.index('(')].strip()
            else:
                param_name = param_part
            if param_name:
                param_descriptions[param_name] = desc

    # Map Python types to JSON schema types
    type_mapping = {
        'str': 'string',
        'int': 'integer',
        'float': 'number',
        'bool': 'boolean',
        'dict': 'object',
        'list': 'array'
    }

    # Build schema
    schema = {
        "type": "function",
        "function": {
            "name": func.__name__,
            "description": description,
            "parameters": {
                "type": "object",
                "properties": {},
                "required": [],
                "additionalProperties": False
            },
            "strict": True
        }
    }

    # Add parameters to schema
    for param_name, param in signature.parameters.items():
        # Get type from type hints, default to str if not specified
        param_type = type_hints.get(param_name, str).__name__
        schema_type = type_mapping.get(param_type, 'string')  # Default to string
        
        schema["function"]["parameters"]["properties"][param_name] = {
            "type": schema_type,
            "description": param_descriptions.get(param_name, f"{param_name} parameter")
        }
        # Mark as required if no default value
        if param.default == inspect.Parameter.empty and param.kind != inspect.Parameter.VAR_KEYWORD:
            schema["function"]["parameters"]["required"].append(param_name)

    return schema

#### Weather function

In [ ]:
def get_weather(location: str) -> dict:
    """
    Get current temperature for a given location.

    Args:
        location (str): City and country e.g. Bogotá, Colombia

    Returns:
        dict: A dictionary containing the current temperature and other weather details.
    """
    return {"location": location, "temperature": 25, "unit": "Celsius"}

In [ ]:
generate_json_schema(func=get_weather)

In [ ]:
tools[0]

#### Tool Calling

In [ ]:
completion = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[{"role": "user", "content": "What is the weather like in Paris today?"}],
    tools=tools
)

In [ ]:
print(completion.choices[0].message.tool_calls)

| 4. Streaming |
|--|


Streaming responses lets you start printing or processing the beginning of the model's output while it continues generating the full response.

In [ ]:
stream = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {
            "role": "user",
            "content": "Say 'double bubble bath' ten times fast.",
        },
    ],
    stream=True,
)

In [ ]:
for chunk in stream:
    print(chunk)
    print(chunk.choices[0].delta)
    print("****************")

| 5. Assistants |
|--|

Build assistants that can call models and use tools to perform tasks.

In [62]:
my_assistant = client.beta.assistants.create(
    instructions="You are a personal math tutor. When asked a question, write and run Python code to answer the question.",
    name="Math Tutor",
    tools=[{"type": "code_interpreter"}],
    model=MODEL_NAME
)

function pass, python function (simple) as tool. Re-use after create

In [63]:
print(my_assistant.id)

asst_ByArr4GEQk8a6p76UoZ5UvXj


In [3]:
my_assistants = client.beta.assistants.list(
    order="desc",
    limit="20",
)

my_assistants.data[0].instructions[:100], my_assistants.data[1].instructions[:100]

('You are a comedian who tells short, family-friendly jokes.',
 '# **Context**  \nYou are **Ava**, a professional and friendly AI chat assistant representing **Amplit')

In [5]:
print(my_assistants.data[1].instructions)

# **Context**  
You are **Ava**, a professional and friendly AI chat assistant representing **Amplity**. Amplity is a full-service partner delivering flexible and specialized medical and commercial services. Amplity supports clients across all stages of the drug lifecycle, scaling with ease to maximize resources and improve impact.  Amplity has five services named as following:
- [Amplity Medical](https://amplity.com/medical)
- [Amplity Sales](https://amplity.com/sales)
- [Amplity Intel](https://amplity.com/intel)
- [Amplity Comms](https://amplity.com/comms)
- [Amplity Learn](https://amplity.com/learn)

Amplity operates the following lines of business: Comms(headed by Susan Duffy), Intel (distinct from Business Intelligence)(headed by Michele Graham), Learn(headed by Michele Graham), Medical(headed by Denise Chambley) and  Sales(headed by Brian O'Donnell). Amplity leaderships also include leaders heading  Business Intelligence, Operations Excellence, Compliance, Technology, Marketting,

| Math Tool |
|--|

In [92]:
def tell_joke(topic: str) -> float:
    """Tells a joke about topic."""
    return f"Why did the chicken say boo about {topic}"

# Define the tool schema for the addition function
tools = {
  "type": "function",
  "function": {
    "name": "tell_joke",
    "description": "Generates a short, family-friendly joke about a specified topic.",
    "parameters": {
      "type": "object",
      "properties": {
        "topic": {
          "type": "string",
          "description": "The topic or subject of the joke (e.g., 'cats', 'space', 'computers')."
        }
      },
      "required": ["topic"],
      "additionalProperties": False
    }
  }
}

In [94]:
client.beta.assistants.update(assistant_id=my_assistant.id, 
    instructions="You are a comedian who tells short, family-friendly jokes.",
    name="Joke Teller",
    model=MODEL_NAME,
    tools=[tools]  # Pass the addition tool
)

Assistant(id='asst_ByArr4GEQk8a6p76UoZ5UvXj', created_at=1748349069, description=None, instructions='You are a comedian who tells short, family-friendly jokes.', metadata={}, model='gpt-4o-mini', name='Joke Teller', object='assistant', tools=[FunctionTool(function=FunctionDefinition(name='tell_joke', description='Generates a short, family-friendly joke about a specified topic.', parameters={'type': 'object', 'properties': {'topic': {'type': 'string', 'description': "The topic or subject of the joke (e.g., 'cats', 'space', 'computers')."}}, 'required': ['topic'], 'additionalProperties': False}, strict=False), type='function')], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=None), top_p=1.0)

Assistant with Tools usage

In [100]:
# Step 1: Create a thread
thread = client.beta.threads.create()
print(f"Created thread with ID: {thread.id}")

Created thread with ID: thread_sfVieYY4xRXeccrYdSVNWGSi


In [101]:
# Step 2: Add a message to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Joke about cats"
)

In [102]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=my_assistant.id
)

In [ ]:
while True:
    run_status = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )
    if run_status.status == "completed":
        break

# Step 5: Retrieve and print the assistant's response
messages = client.beta.threads.messages.list(thread_id=thread.id)
for msg in messages.data:
    print(f"{msg.role.capitalize()}: {msg.content[0].text.value}")

| Threads |
|--|

Create a Thread

In [ ]:
thread = client.beta.threads.create()
thread_id = thread.id
print(f"Thread ID: {thread_id}")

In [ ]:
message = client.beta.threads.messages.create(
    thread_id=thread_id,
    role="user",
    content="Explain factorial?"
)

In [ ]:
run = client.beta.threads.runs.create(
    thread_id=thread_id,
    assistant_id=my_assistant.id
)

# Wait for the run to complete
import time
while True:
    run_status = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run.id)
    if run_status.status == "completed":
        break
    time.sleep(1)

In [ ]:
messages = client.beta.threads.messages.list(thread_id=thread_id)
for message in messages.data:
    if message.role == "assistant":
        print(f"Assistant: {message.content[0].text.value}")

| 6. Response API |
|--|

Allow the model access to external systems and data using function calling.

We get the option to outsource that to OpenAI entirely: you can add a new "store": true property and then in subsequent messages include a "previous_response_id: response_id key to continue that conversation.

In [ ]:
response = client.responses.create(
    model=MODEL_NAME,
    input="Write a one-sentence bedtime story about a unicorn."
)

print(response.output_text)

In [ ]:
response = client.responses.create(
    model=MODEL_NAME,
    tools=[{"type": "web_search_preview"}],
    input="What was a positive news story from today?"
)

print(response.output_text)

Response API with Tools

In [107]:
response = client.responses.create(
  model=MODEL_NAME,
  tools=[{
    "type": "mcp",
    "server_label": "shopify",
    "server_url": "https://pitchskin.com/api/mcp",
  }],
  input="Add the Blemish Toner Pads to my cart"
)

NotFoundError: Error code: 404 - {'error': {'code': '404', 'message': 'Resource not found'}}

In [ ]:
print(response.output_text)

| 7. Response Chaining |
|--|

In [24]:
first_prompt = "Generate a one-sentence idea for a short story about a time traveler."
response1 = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "user", "content": first_prompt}
    ]
)

# Extract the output from the first call
story_idea = response1.choices[0].message.content
print("First response (story idea):", story_idea)

# Second API call: Use the first response to expand into a brief outline
second_prompt = f"Create a brief outline for a short story based on this idea: {story_idea}"
response2 = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "user", "content": second_prompt}
    ]
)

# Extract and print the second response
story_outline = response2.choices[0].message.content
print("\nSecond response (story outline):\n", story_outline)

First response (story idea): When a disillusioned historian discovers a way to travel back to pivotal moments in history, she must choose between altering a tragic event for the better or preserving the timeline, knowing that her decision could erase her own existence.

Second response (story outline):
 **Title:** Echoes of Time

**Outline:**

**I. Introduction**
- **A. Protagonist:** Introduce Rachel, a disillusioned historian specializing in tragic events. She feels her work is futile as history tends to repeat itself.
- **B. Discovery:** Rachel stumbles upon an ancient artifact in a dusty library that allows time travel to pivotal moments in history.
- **C. Motivation:** Driven by personal loss due to a tragic event (e.g., a war or natural disaster), Rachel is tempted to change history for the better.

**II. First Journey: A Test Run**
- **A. Excursion:** Rachel travels to a minor historical event to test the artifact, experiencing the thrill of being a direct observer.
- **B. Conse